In [ ]:
from google.colab import auth
auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!mkdir c-spotpods
#!gcsfuse spotpods /content/c-spotpods
!gcsfuse --implicit-dirs spotpods /content/c-spotpods

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  25115      0 --:--:-- --:--:-- --:--:-- 25115
OK
23 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,278 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 144618 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.30.0_amd64.deb ...
Unpacking gcsfuse (0.30.0) ...
Setting up gcsfuse (0.30.0) ...
Using mount point: /content/c-spotpods
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [ ]:
#Preprocessing
import json
def extract_segments(path):
    """Given path to json file containing an episode extracts all segments of that episode, 
    including start and end time of each segment."""
    with open(path, "r") as read_file:
        episode = json.load(read_file)
    segments=[]
    #had to do "manual" iteration due to irregularities in data
    iter=0
    for segment in episode["results"]:
        seg_result={}
        #make sure there is only one dict in this list (should be true according to dataset description)
        assert len(segment["alternatives"])==1
        segment_dict=segment["alternatives"][0]
        #sometimes "alternatives" dict is empty...
        if "words" and "transcript"  in segment_dict:
            #add segment number
            seg_result["segNum"]=iter
            #add timestamp of the first word in this segment
            seg_result["startTime"]=segment_dict["words"][0]["startTime"]
            #add timestamp of the last word in this segment
            seg_result["endTime"]=segment_dict["words"][-1]["endTime"]
            #add transcript of this segment 
            seg_result["transcript"]=segment_dict["transcript"]
            segments.append(seg_result)
            iter+=1

    return segments

a = extract_segments('c-spotpods')
print(a)

IndentationError: ignored

In [ ]:
!pip install rank_bm25
import json
import rank_bm25 as bm25
import os

podcasts=[]
i=1
for file in os.listdir('c-spotpods'):
  i+=1
  if i == 1000:
    break
  pod=""
  if file == '.ipynb_checkpoints':
    continue
  with open('c-spotpods/' + file) as json_file:
    data=json.load(json_file)
    for lines in data['results']:
      if 'transcript' in lines['alternatives'][0]:
        pod += lines['alternatives'][0]['transcript']
  podcasts.append(pod)

tokenized_podcasts = [pod.split(" ") for pod in podcasts]
bm25model = bm25.BM25Okapi(tokenized_podcasts)

#Here we make a query and then get the best result
query = "joe rogan"
tokenized_query = query.split(" ")

doc_scores = bm25model.get_scores(tokenized_query)
print(bm25model.get_top_n(tokenized_query, podcasts, n=1))

["Hello, welcome again to our interactive new episode. Hope you learned a lot from our last previous listenership. Today. We are going to talk about how to lose weight faster. But safely lose 5 pounds in 1 week. It's a Trope. We see everywhere and while is possible that someone can lose that much if not more in that time period it really depends on your metabolism and loads of of other factors including physical activity and body composition all of which are entirely unique to you weight loss ultimately comes back to the concept of calories in calories out eat less than you burn and you'll lose weight and while is possible to lose water weight quickly on a low-carb diet. I certainly wouldn't advocate for it. The diet itself can trick you into thinking that this eating style is working when really you might gain back what you lost as soon as you eat. Carbs again that can feel incredibly dispiriting if you want results that last longer than a week based on my experience in nutrition coun